# Exploratory Data Analysis (EDA)

## What is EDA?
EDA is an approach to analyzing data typically involving visual methods. It was championed by John W. Tukey in his 1977 book *Exploratory Data Analysis*. He stated that at the time too much emphasis was put on confirmatory data analysis (statistical hypothesis testing). In very simple terms he wanted to look for questions to ask rather than answers to questions.

## Objectives of EDA
- Suggest hypotheses about the causes of observed phenomenon
- Assess assumptions on which statistical inference will be based
- Support the selection of appropriate statistical tools and techniques
- Provide a basis for further data collection

## Graphical Techniques
Some examples of graphical techniques used in EDA include:
- Box (+whisker) Plot
- Histogram
- Run Chart / Time Series plot
- Scatter Plot
- Principal Component Analysis (PCA)

## Getting to know your data
In order to properly visualize your data you first need to get to know your data. There's several different techniques which can allow you to inspect your data to learn more about it.

### Describe
You can use the pandas DataFrame .describe() method to get various summary statistics that exclude NaN values. Let's take a look at the popular Iris dataset

In [1]:
# Import the necessary packages
import pandas as pd
import seaborn as sns
import numpy as np

iris = sns.load_dataset('iris')
iris.describe()

,sepal_length,sepal_width,petal_length,petal_width
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.057333,3.758000,1.199333
std,0.828066,0.435866,1.765298,0.762238
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


### Visual Examination
Another way to get to know your data is to visually inspect this data. Two handy methods are .head and .tail which will return the first or last n rows respectively.

In [2]:
iris.head(5)

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [3]:
iris.tail(5)

,sepal_length,sepal_width,petal_length,petal_width,species
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica
149,5.9,3.0,5.1,1.8,virginica


If you happen to have a very large dataset you can also pull a random n sample of records to visually inspect.

In [4]:
iris.sample(5)

,sepal_length,sepal_width,petal_length,petal_width,species
6,4.6,3.4,1.4,0.3,setosa
51,6.4,3.2,4.5,1.5,versicolor
75,6.6,3.0,4.4,1.4,versicolor
104,6.5,3.0,5.8,2.2,virginica
112,6.8,3.0,5.5,2.1,virginica


### Closer look at your data (Filtering)
#### Query
A simple way to dig deeper into your data is to query that data. Pandas has a handy .query method you can utilize.

In [5]:
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [6]:
# Are there any records which the petal length is larger than the sepal length? (No)
iris.query('petal_length < 1.4')

,sepal_length,sepal_width,petal_length,petal_width,species
2,4.7,3.2,1.3,0.2,setosa
13,4.3,3.0,1.1,0.1,setosa
14,5.8,4.0,1.2,0.2,setosa
16,5.4,3.9,1.3,0.4,setosa
22,4.6,3.6,1.0,0.2,setosa
35,5.0,3.2,1.2,0.2,setosa
36,5.5,3.5,1.3,0.2,setosa
38,4.4,3.0,1.3,0.2,setosa
40,5.0,3.5,1.3,0.3,setosa
41,4.5,2.3,1.3,0.3,setosa


#### [ ] filtering and Boolean Masks
An alternative to query is to utilize a boolean mask. This is where you can apply a numpy-like array of Boolean values to a dataframe to return selected rows. A value of True will return the row and a value of False will skip that row.

In [7]:
# Utilize Pandas broadcasting to return a DataFrame of boolean values
mask = iris['petal_length'] < 1.4
mask.head()

0    False
1    False
2     True
3    False
4    False
Name: petal_length, dtype: bool

In [8]:
# Apply our filtering mask
iris[mask]

,sepal_length,sepal_width,petal_length,petal_width,species
2,4.7,3.2,1.3,0.2,setosa
13,4.3,3.0,1.1,0.1,setosa
14,5.8,4.0,1.2,0.2,setosa
16,5.4,3.9,1.3,0.4,setosa
22,4.6,3.6,1.0,0.2,setosa
35,5.0,3.2,1.2,0.2,setosa
36,5.5,3.5,1.3,0.2,setosa
38,4.4,3.0,1.3,0.2,setosa
40,5.0,3.5,1.3,0.3,setosa
41,4.5,2.3,1.3,0.3,setosa


#### .loc
*Label based indexing*

* Allows us to select data utilizing boolean values or specific column or row names.
* Can also utilize Boolean mask filters in the same fashion as [ ] filtering.

In [9]:
# Always Row, Column
# Return rows 2-5 and display the petal width
# Note: 1,2,3,4 refer to the values within the index and not the row numbers themselves
# - These are the same in this example since we're using the default index which
# -  default to the row numbers.
iris.loc[[1,2,3,4], 'petal_width']

1    0.2
2    0.2
3    0.2
4    0.2
Name: petal_width, dtype: float64

In [10]:
# Display all columns petal_length and beyone with all rows
# Remember Row, Column
iris.loc[:, 'petal_length':].head(5) #.head for readability

,petal_length,petal_width,species
0,1.4,0.2,setosa
1,1.4,0.2,setosa
2,1.3,0.2,setosa
3,1.5,0.2,setosa
4,1.4,0.2,setosa


#### .iloc
*Positional indexing*

* Works the same way as .loc however it only takes boolean values and the integer representation.
* Can also utilize Boolean mask filters in the same fashion as [ ] filtering.

In [11]:
# Get the value of the cell at row 2, column 1.
iris.iloc[1, 0]

4.9000000000000004

In [12]:
# Get the length columns (1 and 3) for the first 5 rows.
iris.iloc[0:5, [0,2]]

,sepal_length,petal_length
0,5.1,1.4
1,4.9,1.4
2,4.7,1.3
3,4.6,1.5
4,5.0,1.4


#### .ix
.ix is a combination of .loc and .iloc which you may see in examples. This can accept either the integer or name for indexes or columns. The difficulty with .ix is the problem if you have an integer based index like in our example. If you had 50 rows which had the value of 10-60 .ix has difficulty if there is an overlap between these values. For example if we did .ix[20,:] does that return the 20th row or does it return the row where the index is 20?

Because of this issue, as of pandas 0.20.0 it has been **deprecated** in favor of .loc or .iloc.

Can also utilize Boolean mask filters in the same fashion as [ ] filtering.

In [13]:
iris.ix[0:5,['sepal_length','petal_length']]

/Users/grantsmith/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


,sepal_length,petal_length
0,5.1,1.4
1,4.9,1.4
2,4.7,1.3
3,4.6,1.5
4,5.0,1.4
5,5.4,1.7


Some of these graphing techniques are outlined under the graphing overview presentation found under the PyData Fort Wayne GitHub repository: https://github.com/PyDataFortWayne/GraphingMatplotlibSeaborn

But before we get into how to graph data let's look at Tidy Data and how to make the data easy to work with.

# Tidy Data
Hadley Wickham published an article in the Journal of Statistical Software called [*Tidy Data*](http://vita.had.co.nz/papers/tidy-data.pdf). In it he outlines characteristics of how to clean data well. A majority of time is spent cleaning data and he wanted to determine how to make data cleaning easy and as effective as possible. It also allows for easier development of tools if the data is in a consistent format.

Tidy Data is defined as:
- Each variable is a column
- Each observation is a row
- Each type of observation unit is a table

As Hadley Wickham comes from an R background which has a lot of support for Tidy Data like the tidyverse package. Currently Python does not but that doesn't mean we can't use other tools at our disposal to make the data tidy.

Let's look at an example. Let's say a researcher is attempting to determine how effective a treatment is. The dataframe may look like this:

In [14]:
df = pd.DataFrame([
            ['John Smith', None, 2.0], 
            ['Jane Doe', 16.0, 11.0], 
            ['Mary Johnson', 3.0, 1.0]
        ], 
        columns=['Patient', 'Treatment A', 'Treatment B']
                  )
df

,Patient,Treatment A,Treatment B
0,John Smith,NaN,2.0
1,Jane Doe,16.0,11.0
2,Mary Johnson,3.0,1.0


However, this same data could be represented in a different format but still project the same information. For example:

In [15]:
pd.DataFrame([
                ['Treatment A', None, 16.0, 3.0], 
                ['Treatment B', 2.0, 11.0, 1.0]
             ], 
             columns=['Treatment', 'John Smith', 'Jane Doe', 'Mary Johnson'])

,Treatment,John Smith,Jane Doe,Mary Johnson
0,Treatment A,NaN,16.0,3.0
1,Treatment B,2.0,11.0,1.0


This is what Hadley Wickham is attempting to solve with Tidy Data. That same data represented in Tidy format would look like this:

In [16]:
df_tidy = pd.DataFrame([['John Smith', 'a', None],
                        ['Jane Doe', 'a', 16.0],
                        ['Mary Johnson', 'a', 3.0],
                        ['John Smith', 'b', 2.0],
                        ['Jane Doe', 'b', 11.0],
                        ['Mary Johnson', 'b', 1.0]
                       ], 
                       columns=['Patient Name', 'Treatment', 'Result']
                      )
df_tidy

,Patient Name,Treatment,Result
0,John Smith,a,NaN
1,Jane Doe,a,16.0
2,Mary Johnson,a,3.0
3,John Smith,b,2.0
4,Jane Doe,b,11.0
5,Mary Johnson,b,1.0


# Cleaning Data
## Melt
https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.melt.html


“Unpivots” a DataFrame from wide format to long format, optionally leaving identifier variables set.

As a reminder, the treatment table from earlier looked like this:

In [17]:
df

,Patient,Treatment A,Treatment B
0,John Smith,NaN,2.0
1,Jane Doe,16.0,11.0
2,Mary Johnson,3.0,1.0


If we apply the melt method to this DataFrame we can make it Tidy:

In [18]:
melted_df = df.melt(id_vars='Patient', value_name='Result', var_name='Treatment')
melted_df

,Patient,Treatment,Result
0,John Smith,Treatment A,NaN
1,Jane Doe,Treatment A,16.0
2,Mary Johnson,Treatment A,3.0
3,John Smith,Treatment B,2.0
4,Jane Doe,Treatment B,11.0
5,Mary Johnson,Treatment B,1.0


## Pivot
https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.pivot.html

Reshape data (produce a “pivot” table) based on column values. Uses unique values from index / columns to form axes of the resulting DataFrame.

In [19]:
melted_df

,Patient,Treatment,Result
0,John Smith,Treatment A,NaN
1,Jane Doe,Treatment A,16.0
2,Mary Johnson,Treatment A,3.0
3,John Smith,Treatment B,2.0
4,Jane Doe,Treatment B,11.0
5,Mary Johnson,Treatment B,1.0


In [20]:
melted_df.pivot(index='Patient', columns='Treatment', values='Result').reset_index()

Treatment,Patient,Treatment A,Treatment B
0,Jane Doe,16.0,11.0
1,John Smith,NaN,2.0
2,Mary Johnson,3.0,1.0


In [21]:
df

,Patient,Treatment A,Treatment B
0,John Smith,NaN,2.0
1,Jane Doe,16.0,11.0
2,Mary Johnson,3.0,1.0


## Splitting Fields
Sometimes it's necessary to split fields into different columns. Luckily this is fairly easy with pandas. Let's cleanup our melted dataset and cleanup the treatment value field. We can split the field and use just the treatment letter.

In [22]:
# pat - What to split on
# expand=True - Expand out the list to a DataFrame
# [1] - Select the second column of tha dataframe
melted_df['Treatment'] = melted_df['Treatment'].str.split(pat=' ', expand=True)[1]
melted_df

,Patient,Treatment,Result
0,John Smith,A,NaN
1,Jane Doe,A,16.0
2,Mary Johnson,A,3.0
3,John Smith,B,2.0
4,Jane Doe,B,11.0
5,Mary Johnson,B,1.0


A more complex example looking at Apache access logs which wasn't recorded in a nice simple csv format but rather a specific format. We can use a complex regular expression to split out the fields into a dataframe to ease parsing and analysis.

In [23]:
import os.path
import re

# Read in sample log file
df_apache = pd.read_csv(os.path.join('data', 'apache_access.txt'), 
                        header=None, 
                        names=['RAW'])

# Display entire contents of cells
pd.set_option('display.max_colwidth', 2000)
df_apache.head()

,RAW
0,"127.0.0.1 - - [07/Mar/2004:16:05:49 -0800] ""POST /twiki/bin/edit/Main/Double_bounce_sender?topicparent=Main.ConfigurationVariables HTTP/1.1"" 401 12846"
1,"127.0.0.1 - - [07/Mar/2004:16:06:51 -0800] ""POST /twiki/bin/rdiff/TWiki/NewUserTemplate?rev1=1.3&rev2=1.2 HTTP/1.1"" 200 4523"
2,"127.0.0.1 - - [07/Mar/2004:16:10:02 -0800] ""POST /mailman/listinfo/hsdivision HTTP/1.1"" 200 6291"
3,"127.0.0.1 - - [07/Mar/2004:16:11:58 -0800] ""GET /twiki/bin/view/TWiki/WikiSyntax HTTP/1.1"" 200 7352"
4,"127.0.0.1 - - [07/Mar/2004:16:20:55 -0800] ""GET /twiki/bin/view/Main/DCCAndPostFix HTTP/1.1"" 200 5253"


In [24]:
# Define regular expression to parse each line
apache_regex = r'^(?P<ip_address>(?:\d{1,3}\.){3}\d{1,3}) ' + \
               r'[^ ]* [^ ]* \[(?P<request_time>[^\]]*)\] ' + \
               r'"(?P<method>[^ ]*) ?(?P<url>[^ ]*) ' + \
               r'(?P<http_version>HTTP\/\d\.\d)" ' + \
               r'(?P<status_code>\d+) ' + \
               r'(?P<apache_pid>\d+)$'
are = re.compile(apache_regex)

# Split the fields into new columns in a new dataframe
df_split = df_apache['RAW'].str.split(apache_regex, expand=True)

# Remove empty columns
del df_split[0]
# Alternatively...
df_split = df_split.drop(8, axis=1)

# Label the columns from the regular expression
df_split.columns = are.groupindex.keys()

df_split.head()

,ip_address,request_time,method,url,http_version,status_code,apache_pid
0,127.0.0.1,07/Mar/2004:16:05:49 -0800,POST,/twiki/bin/edit/Main/Double_bounce_sender?topicparent=Main.ConfigurationVariables,HTTP/1.1,401,12846
1,127.0.0.1,07/Mar/2004:16:06:51 -0800,POST,/twiki/bin/rdiff/TWiki/NewUserTemplate?rev1=1.3&rev2=1.2,HTTP/1.1,200,4523
2,127.0.0.1,07/Mar/2004:16:10:02 -0800,POST,/mailman/listinfo/hsdivision,HTTP/1.1,200,6291
3,127.0.0.1,07/Mar/2004:16:11:58 -0800,GET,/twiki/bin/view/TWiki/WikiSyntax,HTTP/1.1,200,7352
4,127.0.0.1,07/Mar/2004:16:20:55 -0800,GET,/twiki/bin/view/Main/DCCAndPostFix,HTTP/1.1,200,5253


Alternatively you can use the extract method which is much cleaner when using a regular expression

In [25]:
df_extracted = df_apache['RAW'].str.extract(apache_regex, expand=True)
df_extracted.head()

,ip_address,request_time,method,url,http_version,status_code,apache_pid
0,127.0.0.1,07/Mar/2004:16:05:49 -0800,POST,/twiki/bin/edit/Main/Double_bounce_sender?topicparent=Main.ConfigurationVariables,HTTP/1.1,401,12846
1,127.0.0.1,07/Mar/2004:16:06:51 -0800,POST,/twiki/bin/rdiff/TWiki/NewUserTemplate?rev1=1.3&rev2=1.2,HTTP/1.1,200,4523
2,127.0.0.1,07/Mar/2004:16:10:02 -0800,POST,/mailman/listinfo/hsdivision,HTTP/1.1,200,6291
3,127.0.0.1,07/Mar/2004:16:11:58 -0800,GET,/twiki/bin/view/TWiki/WikiSyntax,HTTP/1.1,200,7352
4,127.0.0.1,07/Mar/2004:16:20:55 -0800,GET,/twiki/bin/view/Main/DCCAndPostFix,HTTP/1.1,200,5253


## Standardizing Fields
In order to help our machine learning algorithms work properly we need to standardize the field values. One main way of doing this is applying a function to a column. You can define your own function or for simpler operations utilize a lambda function.

Take the following code as an example.

In [26]:
tips = sns.load_dataset('tips')

def get_gender_code(gender_value):
    if gender_value == 'Male':
        return 1
    else:
        return 0


tips['gender_code'] = tips['sex'].apply(get_gender_code)
tips.head()

,total_bill,tip,sex,smoker,day,time,size,gender_code
0,16.99,1.01,Female,No,Sun,Dinner,2,0
1,10.34,1.66,Male,No,Sun,Dinner,3,1
2,21.01,3.50,Male,No,Sun,Dinner,3,1
3,23.68,3.31,Male,No,Sun,Dinner,2,1
4,24.59,3.61,Female,No,Sun,Dinner,4,0


Here is the same example using a lambda function:

In [27]:
tips['gender_code_lambda'] = tips['sex'].apply(lambda x: 1 if x=='Male' else 0)
tips.head()

,total_bill,tip,sex,smoker,day,time,size,gender_code,gender_code_lambda
0,16.99,1.01,Female,No,Sun,Dinner,2,0,0
1,10.34,1.66,Male,No,Sun,Dinner,3,1,1
2,21.01,3.50,Male,No,Sun,Dinner,3,1,1
3,23.68,3.31,Male,No,Sun,Dinner,2,1,1
4,24.59,3.61,Female,No,Sun,Dinner,4,0,0


In [28]:
days = {'Thur':'Thursday','Fri':'Friday','Sat':'Saturday','Sun':'Sunday'}
tips['day_name'] = tips['day'].map(days)
print(tips['day'].unique())
print(tips['day_name'].unique())

[Sun, Sat, Thur, Fri]
Categories (4, object): [Sun, Sat, Thur, Fri]
['Sunday' 'Saturday' 'Thursday' 'Friday']


## Categorical Data
https://pandas.pydata.org/pandas-docs/stable/categorical.html

Pandas can implement a Categorical data type which is simply a predefined list of accepted values. This allows us to use less memory when processing our data.

Let's take a look at the method column from our apache example above.

In [29]:
df_split['method'].value_counts()

GET     24
POST     3
Name: method, dtype: int64

In [30]:
df_split['method'].memory_usage()

296

In [31]:
df_split['method'] = df_split['method'].astype('category')
df_split['method'].memory_usage()

203

## Handling Missing Data
There are several different options you have available to deal with missing values. Let's look at an example datafame.

In [32]:
df = pd.DataFrame([[np.nan, 2, np.nan, 0],
                    [3, 4, np.nan, 1],
                    [np.nan, np.nan, np.nan, 5],
                    [np.nan, 3, np.nan, 4]],
                    columns=list('ABCD'))
df

,A,B,C,D
0,NaN,2.0,NaN,0
1,3.0,4.0,NaN,1
2,NaN,NaN,NaN,5
3,NaN,3.0,NaN,4


### IsNull
When doing EDA you might want to dig deeper into your data which has null values. Pandas provides a helpful .isnull() function which you can use on either a DataFrame of Series level. It will return either a True or False value for each element.

In [33]:
df.isnull()

,A,B,C,D
0,True,False,True,False
1,False,False,True,False
2,True,True,True,False
3,True,False,True,False


We can utilize this array as a mask to filter records we're interested in. Let's take a look at all records where A is null.

In [34]:
mask = df['A'].isnull()
df[mask]

,A,B,C,D
0,NaN,2.0,NaN,0
2,NaN,NaN,NaN,5
3,NaN,3.0,NaN,4


### Drop
If you don't care about the NaN values you can just drop them.

In [35]:
# Drop columns (axis=1) where ALL records are missing
df.dropna(axis=1, how='all')

,A,B,D
0,NaN,2.0,0
1,3.0,4.0,1
2,NaN,NaN,5
3,NaN,3.0,4


In [36]:
# Drop columns where ANY records are missing
df.dropna(axis=1, how='any')

,D
0,0
1,1
2,5
3,4


### Fill
There's several different ways to fill the missing data within pandas. You should be very mindful of how you fill missing values as it will add a certain amount of bias to your data which could affect the performance of your algorithms.

Let's take a look at a few of them.

#### Fill NA
Replace all empty values with a supplied value.

In [37]:
df.fillna(0)

,A,B,C,D
0,0.0,2.0,0.0,0
1,3.0,4.0,0.0,1
2,0.0,0.0,0.0,5
3,0.0,3.0,0.0,4


#### Forward Fill
Fill empty values with the *previous* non-null value

In [38]:
df.fillna(method='ffill')

,A,B,C,D
0,NaN,2.0,NaN,0
1,3.0,4.0,NaN,1
2,3.0,4.0,NaN,5
3,3.0,3.0,NaN,4


#### Backfill
Fill empty values with the *next* non-null value.

In [39]:
df.fillna(method='bfill')

,A,B,C,D
0,3.0,2.0,NaN,0
1,3.0,4.0,NaN,1
2,NaN,3.0,NaN,5
3,NaN,3.0,NaN,4


#### Interpolation
Fill empty values with a "guess" based on linear interpolation.

In [40]:
df.interpolate()

,A,B,C,D
0,NaN,2.0,NaN,0
1,3.0,4.0,NaN,1
2,3.0,3.5,NaN,5
3,3.0,3.0,NaN,4


## Combining Data
Typically as a part of EDA we will need to combine two different data sources. Let's take these two simple datafames and combine them one after the other.

In [41]:
df1 = pd.DataFrame({'a':[1,2,3]})
df1

,a
0,1
1,2
2,3


In [42]:
df2 = pd.DataFrame({'a':[10,9,8]})
df2

,a
0,10
1,9
2,8


### Append
To combine both dataframes on top of each other you can utilize the append method.

In [43]:
df1.append(df2)

,a
0,1
1,2
2,3
0,10
1,9
2,8


### Concat
We can similarly utilize the .concat() function to do the same as .append() by specifying an axis.

In [44]:
pd.concat([df1, df2], axis=0)

,a
0,1
1,2
2,3
0,10
1,9
2,8


One added benefit of utilizing concat is we can combine dataframes on a column basis as well (side by side) by changing the axis to be 1 (for columns).

In [45]:
pd.concat([df1, df2], axis=1)

,a,a
0,1,10
1,2,9
2,3,8


### Merge
Many times you'll need to combine database tables based on certain criteria or column(s) similar to the JOIN command in SQL. In this next example there are two different tables - Employees and Salaries. Employee is indexed by the emp_id field whereas the Salary table is using a default index. (Ideally the Salary table is indexed by the employee id as well but for this example they are not.)

In [46]:
df_emp = pd.DataFrame({'emp_id':[100,101,102,103,104], 'Name':['Donald Trump','Hillary Clinton','Jill Stein','Gary Johnson','Mickey Mouse']})
df_emp = df_emp.set_index('emp_id')
df_emp

,Name
emp_id,
100,Donald Trump
101,Hillary Clinton
102,Jill Stein
103,Gary Johnson
104,Mickey Mouse


In [47]:
df_salary = pd.DataFrame({'emp_id':[100,101,102,103], 'salary':[1000000000, 123456, 56789, 3.1415926535]})
df_salary

,emp_id,salary
0,100,1.000000e+09
1,101,1.234560e+05
2,102,5.678900e+04
3,103,3.141593e+00


To join the employees to their salaries we utilize the .merge() command. We specify the left and right tables along with how to merge the two tables (and if we're using an index or not). The result is a list of employees who have a salary (Mickey Mouse is missing since he's not paid).

In [48]:
pd.merge(df_emp, df_salary, left_index=True, right_index=False, right_on='emp_id')

,Name,emp_id,salary
0,Donald Trump,100,1.000000e+09
1,Hillary Clinton,101,1.234560e+05
2,Jill Stein,102,5.678900e+04
3,Gary Johnson,103,3.141593e+00


Alternatively if we wanted to do a left join and make sure Mickey Mouse is included even though he doesn't have a salary we cantilize the *how* parameter and set it to 'left'.

In [49]:
pd.merge(df_emp, df_salary, left_index=True, right_index=False, right_on='emp_id', how='left')

,Name,emp_id,salary
0,Donald Trump,100,1.000000e+09
1,Hillary Clinton,101,1.234560e+05
2,Jill Stein,102,5.678900e+04
3,Gary Johnson,103,3.141593e+00
3,Mickey Mouse,104,NaN
